In [176]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

### Récuperation de la liste des plateformes de jeu 

In [106]:
r = requests.get('https://www.dureedevie.fr')
request_text = r.text
soup = BeautifulSoup(request_text)
plateformes = []
for a in soup.find_all("a", {"class": "text-lg text-gray-900 font-medium hover:text-gray-600"}) :
    plateformes.append([a.text, a.get('href')])


In [107]:
plateformes

[['Nos conseils jeux pour Noël', '/piles/nos-conseils-jeux-pour-noel'],
 ['Game Awards 2021', '/piles/game-awards-2021'],
 ['XBOX Game Pass', '/piles/xbox-game-pass'],
 ['Apple Arcade', '/piles/apple-arcade'],
 ['Attentes 2021', '/piles/attentes-2021'],
 ['Attentes 2022', '/piles/attentes-2022'],
 ['Bandes originales', '/piles/bandes-originales'],
 ['Gamekult Awards 2020', '/piles/gamekult-awards-2020'],
 ['Pégases 2021', '/piles/pegases-2021'],
 ['Game Awards 2020', '/piles/game-awards-2020'],
 ['Pégases 2020', '/piles/pegases-2020'],
 ["Pixel d'Or 2019", '/piles/pixel-d-or-2019'],
 ['Game Awards 2019', '/piles/game-awards-2019'],
 ['Android', '/plateformes/android'],
 ['Game Boy Advance', '/plateformes/game-boy-advance'],
 ['Game Boy Color', '/plateformes/game-boy-color'],
 ['Google Stadia', '/plateformes/google-stadia'],
 ['iPhone OS (iOS)', '/plateformes/iphone-os-ios'],
 ['Linux', '/plateformes/linux'],
 ['Mac', '/plateformes/mac'],
 ['Megadrive', '/plateformes/megadrive'],
 ['NES

### Suppression des classements

In [108]:
for i in range(0,13,1):
    plateformes.pop(0)

In [109]:
plateformes

[['Android', '/plateformes/android'],
 ['Game Boy Advance', '/plateformes/game-boy-advance'],
 ['Game Boy Color', '/plateformes/game-boy-color'],
 ['Google Stadia', '/plateformes/google-stadia'],
 ['iPhone OS (iOS)', '/plateformes/iphone-os-ios'],
 ['Linux', '/plateformes/linux'],
 ['Mac', '/plateformes/mac'],
 ['Megadrive', '/plateformes/megadrive'],
 ['NES', '/plateformes/nes'],
 ['Nintendo 3DS', '/plateformes/nintendo-3ds'],
 ['Nintendo 64', '/plateformes/nintendo-64'],
 ['Nintendo DS', '/plateformes/nintendo-ds'],
 ['Nintendo Gamecube', '/plateformes/nintendo-gamecube'],
 ['Nintendo Switch', '/plateformes/nintendo-switch'],
 ['Oculus Quest', '/plateformes/oculus-quest'],
 ['PC', '/plateformes/pc'],
 ['PC VR', '/plateformes/pc-vr'],
 ['PlayStation', '/plateformes/playstation'],
 ['PlayStation 2', '/plateformes/playstation-2'],
 ['PlayStation 3', '/plateformes/playstation-3'],
 ['PlayStation 4', '/plateformes/playstation-4'],
 ['PlayStation 5', '/plateformes/playstation-5'],
 ['PlayS

### récuperation de tous les jeux sur toutes les plateformes

In [238]:
%%time
listeJeux = []
#recuperation de la liste des jeux pour toutes les plateformes
for plateforme in plateformes :
    r = requests.get('https://www.dureedevie.fr{}'.format(plateforme[1]))
    request_text = r.text
    soup = BeautifulSoup(request_text)
    #pour chaque jeu
    for a in soup.find_all("a", {"class": "underline"}) :
        r = requests.get('https://www.dureedevie.fr{}'.format(a.get('href')))
        request_text = r.text
        soup = BeautifulSoup(request_text)
        data = []
        for dd in soup.find_all("dd"):
            data.append(dd.text)
        while len(data) <9 :
            data.insert(0,'N/A')
        if len(data) == 9 : 
            listeJeux.append([a.text ,plateforme[0], data])
        #print(listeJeux)

Wall time: 3min 12s


In [239]:
len(listeJeux)

2184

In [250]:
listeJeux[0][2]

['N/A',
 'N/A',
 'N/A',
 'N/A',
 'Course/Conduite',
 'Nintendo Switch, Android, iPhone OS (iOS)',
 'Pixel Perfect Dude, Lionsharp Studios',
 'Pixel Perfect Dude, PM Studios',
 '20/03/2019']

In [251]:
d = { 'titre': [listeJeux[i][0] for i in range(len(listeJeux))],
    'Temps de jeu moyen' : [listeJeux[i][2][0] for i in range(len(listeJeux))],
     'Histoire  Trame principale' : [listeJeux[i][2][1] for i in range(len(listeJeux))],
     'Histoire + Quêtes secondaires' : [listeJeux[i][2][2] for i in range(len(listeJeux))],
     'Complétionniste (100%)' : [listeJeux[i][2][3] for i in range(len(listeJeux))],
     'Genres' : [listeJeux[i][2][4] for i in range(len(listeJeux))],
     'Plateformes' : [listeJeux[i][1] for i in range(len(listeJeux))],
     'Développeur' : [listeJeux[i][2][6] for i in range(len(listeJeux))],
     'Editeur' : [listeJeux[i][2][7] for i in range(len(listeJeux))],
     'Date de sortie' : [listeJeux[i][2][8] for i in range(len(listeJeux))]
    }
df = pd.DataFrame(data=d)
df.to_csv(r'dataframe.csv',sep=';')

## Clean du csv

### Converstion String to Datetime

In [ ]:
df['Date de sortie'] = pd.to_datetime(df['Date de sortie'], format='%d/%m/%Y',errors='coerce')

## Analytics

### Nombres de jeux par plateformes

In [268]:
df['Plateformes'].value_counts()

PC                   546
PlayStation 4        362
Nintendo Switch      322
Xbox One             302
PlayStation 5        128
Xbox Series X/S      123
iPhone OS (iOS)       94
Mac                   66
Android               58
Google Stadia         41
PlayStation 3         24
PlayStation Vita      20
Xbox 360              18
Linux                 16
Oculus Quest          13
PlayStation VR        11
Wii U                  7
PC VR                  7
Nintendo 3DS           7
Wii                    4
Xbox                   4
PlayStation 2          3
Nintendo DS            2
Nintendo Gamecube      2
PlayStation            1
Megadrive              1
Nintendo 64            1
Saturn                 1
Name: Plateformes, dtype: int64

In [275]:
print (df.dtypes)

titre                            object
Temps de jeu moyen               object
Histoire  Trame principale       object
Histoire + Quêtes secondaires    object
Complétionniste (100%)           object
Genres                           object
Plateformes                      object
Développeur                      object
Editeur                          object
Date de sortie                   object
dtype: object


In [292]:
df['Date de sortie'].dt.year.value_counts()

2021.0    995
2020.0    340
2019.0    267
2018.0    144
2016.0     90
2017.0     83
2015.0     49
2013.0     45
2014.0     32
2011.0     22
2010.0     22
2009.0     20
2012.0     18
2008.0     14
2003.0     10
1998.0      7
2007.0      5
2006.0      5
1996.0      3
2004.0      2
2002.0      1
Name: Date de sortie, dtype: int64

In [280]:
df['Date de sortie']

0      2019-03-20
1      2015-02-25
2      2018-11-16
3      2016-03-03
4      2019-06-18
          ...    
2179   2021-07-06
2180   2021-02-04
2181   2020-12-01
2182   2020-03-06
2183   2021-09-30
Name: Date de sortie, Length: 2184, dtype: datetime64[ns]